In [18]:
from dotenv import load_dotenv
import sys

sys.path.append("../common")
load_dotenv()

True

In [19]:
import os
from langsmith_tracker import set_tracking
from langchain_print import stream_response
from multimodal import MultiModal
from tools_news import GoogleNews

# 인스턴스를 생성할 때 필요한 매개변수를 전달합니다.
set_tracking(project_name="15.Agent, Agentic RAG")

Langsmith 추적이 활성화되었습니다. [프로젝트명: 15.Agent, Agentic RAG]


### Agentic RAG

Agent 로 RAG 를 진행합니다.  
예를 들면, 파일시스템을 사용하여 파일의 내용을 보정하거나 csv 데이터를 활용하여 데이터 분석을 하기도 합니다.  
이메일 자동 답변 에이전트, 이메일 발송 등의 에이전트 구현이 가능합니다. 이외에도 사용자가 원하는 에이전트를 구현 할 수 있게 합니다.  
  
Agent RAG를 사용하면 `LLM + 도구` 를 사용한다는 점이 기존의 RAG 와는 다른 점입니다. (Agent RAG가 이점은 있지만 모든 상황에서 유리한 것은 아닙니다. RAG만 사용해도 될 경우는 RAG만 사용하는 것이 좋습니다.)  
예를 들면, retirever(검색기) + (웹검색 + 코드작성 + ...)  

#### 예제
웹검색 도구, 코드 생성 도구를 사용하여 Agent RAG 를 구현합니다.

#### 웹 검색 도구

In [28]:
from langchain_community.tools.tavily_search import TavilySearchResults

# search 라는 변수로 TavilySearchResults를 선언
# k : 검색 결과 수
search = TavilySearchResults(name="web_search", k=5)

`search.invoke()` 함수는 주어진 문자열에 대한 검색을 실행합니다.  

In [9]:
search.invoke("2024년 스타벅스 프리퀀시 상품은 무엇인가요?")

[{'url': 'https://kgon.tistory.com/entry/2024년-스타벅스-프리퀀시-적립-방법과-팁-2025플래너-캘린더-포터블-램프-24-WINTER-e-FREQUENCY-STARBUCKS-PORTABLE-LAMP-크리스마스-프로모션-음료-3잔-리저브-카테고리-음료',
  'content': '2024년 스타벅스 프리퀀시 적립 방법과 팁 - 2025플래너 캘린더 포터블 램프 ( 24 winter e-frequency ) ( starbucks portable lamp ) [ 크리스마스 프로모션 음료 3잔 + 리저브 카테고리 음료 ]1. ... • 품절 대비: 프리퀀시 기획 상품은 조기 품절이 될 수 있으니, 빠르게 교환하는'},
 {'url': 'https://m.blog.naver.com/kmin046/223665366076',
  'content': '(스타벅스) 2024년 스벅 겨울 e-프리퀀시 쉽게 모으기 (프리퀀시 소개/팁, 방법/예약/후기) ... 이번에 스타벅스 e-프리퀀시의 상품은 저도 아주 탐이 나더라구요! ... #스타벅스 #스타벅스프리퀀시 #스타벅스e #스타벅스e프리퀀시 #스벅 #스벅프리퀀시 #스타벅스'},
 {'url': 'https://thanksposting.com/entry/2024-스타벅스-프리퀀시-겨울-증정품-8종플래너-캘린더-포터블램프',
  'content': '2024년 스타벅스 겨울 e-프리퀀시 이벤트가 돌아왔습니다! 올해는 어떤 증정품들이 준비되어 있는지 확인해 보겠습니다. 프리퀀시 참여 방법에 대한 정보도 함께 공유하겠습니다. ≣ 목차 e-프리퀀시 증정품 소개 ☑️2024 겨울 e-프리퀀시 증정품과 디지털 혜택 이번 겨울 시즌 e-프리퀀시는 오랜 시간'},
 {'url': 'https://m.post.naver.com/viewer/postView.naver?volumeNo=39791101&memberNo=25041664',
  'content': "스타벅스 코리아가 11월 1일부터 12월 31일까지 특

### 문서 기반 문서 검색 도구 : Retriever

예제에 사용할 문서는 data 폴더 하위에 넣습니다.   
  
아래 예제 코드는 웹 기반 문서 로더, 문서 분할기, 벡터 저장소, 그리고 OpenAI 임베딩을 사용하여 문서 검색 기능을 구축합니다.  
PDF 문서를 FAISS 메모리 저장소에 저장하고 조회하는 retirever를 생성합니다.

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드
loader = PyPDFLoader("./data/IS-183_AI 위험 유형 및 사례 분석(최종).pdf")

# 텍스트 분할기 선언
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서 로드 후 분할
split_docs = loader.load_and_split(text_splitter)

# VectorStore 생성
vector = FAISS.from_documents(documents=split_docs, embedding=OpenAIEmbeddings())

# Retriever 생성
retriever = vector.as_retriever()

`retiriever.invoke()` 함수를 사용하여 사용자의 질문에 대해 **가장 관련성이 높은 문서**를 찾습니다.

In [11]:
# 문서에서 관련성 높은 문서 찾기
retriever.invoke("네이버가 개발하 생성형 AI 관련 내용을 문서에서 찾아주세요.")

[Document(metadata={'source': './data/IS-183_AI 위험 유형 및 사례 분석(최종).pdf', 'page': 28}, page_content='47 과학기술정보통신부 (2024.10), “AI안전연구소 설립·운영계획”, 보도자료'),
 Document(metadata={'source': './data/IS-183_AI 위험 유형 및 사례 분석(최종).pdf', 'page': 21}, page_content='ㅇ KB국민, 하나, 신한은행 등 금융 분야를 비롯하여 다양한 분야에서의 AI 상담 서비스(콜센터)의 도입으로 AI가 상담사의 역할을 대체 (2024.2.17)38ㅇ 개발자의 단순 코딩 업무 대체, 제조 분야에서의 공정 자동화, 문서 작성 등 AI는 높은 생산성을 제공하여 사람보다 더 효율적인 작업이 가능 (2024.2.17)ㅇ 포스코는 무인로봇·비전AI 등을 활용한 공정 자동화로 인텔리전트 팩토리로의 전환을 실현하여 최소한의 인력만 유지39 (2024.6.18)사진: American Banker(2018)40■(글로벌AI\x00격차)\x00고급AI\x00개발자원의보유여부에따른국가간격차발생●고급 AI 개발을 위한 다양한 필요조건들로 인해 AI 빅테크 기업들의 지배력 확대∙디지털 기술 활용률, 컴퓨팅 자원 접근성, 인프라, 경제적 의존도 등은 범용 AI 개발 및 배포에 있어 중요한 요소로서, 이로 인해 범용 AI의 개발은 현재 서구 국가 및 중국에 집중된 경향*특히 미국에 기반을 둔 소수 기업들이 시장을 선도하며, 가치 및 문화 편향, 공급망 불평등 등 글로벌 격차가 야기되는 상황∙숙련된 인재 집중도의 불균형과 개발 및 유지에 드는 막대한 재정적 비용은 글로벌 AI 격차를 더욱 심화●AI에 관한 연구는 미국과 중국이 주도하고 있으며, 주요 AI 선도국들은 연구 및 자원 보유 여부에 따라 AI 산업의 영향력을 보유∙기계 학습 모델 개발은 미국, 캐나다, 영국 및 중국 등 일부 국가에 집중되어 있고, 대규모 언어 및 멀티모달 모

위에 만든 `retirever`를 도구로 변환시키기 위해 `create_retriever_tool()` 함수를 사용합니다.

In [26]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever, name="pdf_search", description="use this tool to search in a pdf file") # 도구에 대한 설명을 자세히 기입!!

#### Agent가 사용할 도구 목록 정의

두 가지 도구를 생성했으므로, Agent 가 사용할 도구 목록(tools)을 생성합니다.  
`tools` 라는 리스트는 `search` 와 `retrieval` 을 포함합니다.  

In [27]:
# tools 라는 list 형 변수에 search와 retrieval_tool 넣음
tools = [search, retriever_tool]

### Agent 생성

Agent가 사용할 도구가 생성되었으니 Agent를 구현할 차례입니다.  
Agent 는 LLM + Tools + Prompt 이기 때문에 이제 LLM과 Prompt 를 정의할 일이 남아있습니다.  
LLM을 생성하고 Prompt를 정의합니다.  

In [45]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# LLM 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information form the PDF document."
            "If you can't find the exact iinformation from the PDF document, please use the `web_search` tool for searching information form the web."
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{user_input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

다음으로 LLM 에 tools 를 바인딩하는 `Tool Calling Agent` 를 생성합니다.  
이 단계에서 Agent가 생성되는 것 입니다.

In [41]:
from langchain.agents import create_tool_calling_agent

# Agent 생성, tool calling agent
agent = create_tool_calling_agent(llm, tools, prompt)

생성된 Agent를 실행하기 위해서는 `AgentExecutor` 가 필요합니다.  
`AgentExecutor` 를 생성합니다. 

In [42]:
from langchain.agents.agent import AgentExecutor 

# AgentExecutor 생성
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False) # verbose 는 중간단계 출력을 위한 설정 값

#### Agent 실행 하기

몇 가지 질의에 대해 에이전트를 실행할 수 있습니다.  
현재 이러한 셀에서 작성한 모든 에이전트는 **상태(Stateless)가 없는 질의**입니다. 이전 상호작용을 기억하지 않습니다.  
  

agent_executors는 invoke(), stream() 함수를 통해 딕셔너리 인자를 받아 실행합니다.  

In [20]:
from langchain_print import AgentStreamParser

# 각 단계별 출력을 위한 파서 생성
agent_stream_parser = AgentStreamParser()

아래는 web_search 도구를 선택할만한 질의를 한 예제입니다.

In [46]:

# AgentExecutor 실행
result = agent_executor.stream(
    {
        "user_input": "스타벅스의 2024년 winter 프리퀀시의 미션음료와 증정품은 무엇인가요?"
    } # 구어체로 input을 주어도 아래 query 를 확인하면 정확한 키워드를 찾아서 검색을 시도
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별 출력
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 스타벅스 2024년 winter 프리퀀시 미션음료 증정품
Log: 
Invoking: `tavily_search_results_json` with `{'query': '스타벅스 2024년 winter 프리퀀시 미션음료 증정품'}`



[관찰 내용]
Observation: [{'url': 'https://m.post.naver.com/viewer/postView.naver?volumeNo=40042593&memberNo=1758487', 'content': '2024년 스타벅스 겨울 프리퀀시 이벤트 증정품 3개 및 미션음료 참가방법 ... 스타벅스 2024 winter e-frequency 이벤트는 11월 1일부터 12월 31일까지 진행됩니다. 겨울 시즌 동안 따뜻한 스타벅스 음료와 함께 특별한 증정품을 즐기며 추운 겨울을 감성적으로 보내 보세요.'}, {'url': 'https://hobakkoong.tistory.com/163', 'content': '2024년 스타벅스 윈터 e-프리퀀시 (증정품- 다이어리, 캘린더, 펜), 미션음료 종류 *구성 : 스타벅스 데일리 플래너 2종 - 카멜, 샌드\xa0 \xa0 \xa0 \xa0 \xa0 스타벅스 언데이티드 플래너 1종 - 블랙\xa0이번 2024년 스타벅스 다이어리에는 음료 지류쿠폰이\xa0포함되어 있지 않다고 하네요(증정용only)\xa011월2일부터 6일까지 5일간 더블 플래너이벤트도 진행한다고 합니다. ♣ e-프리퀀시 이벤트 유의사항 ◎ 참여방법 : 스타벅스 모바일 APP 또는 홈페이지(PC 또는 모바일)에 등록된 e-프리퀀시 바코드 화면또는 스타벅스 카드를 매장파트너에게 제시하시면제조 음료 구매시 e-스티커를 적립가능\xa0◎ 2024년 1월 1일부터 사용하지 않은 e-프리퀀시\xa0증정품 교환 e-쿠폰, e-스티커, 예약확인증은모두 소멸됩니다. ◎\xa0 당일예약(예약 완료일과 수령일이 동일한 경우)에 한해 매장 

아래는 pdf_search 도구를 선택할만한 질의를 한 예제입니다.

In [48]:
# AgentExecutor 실행
result = agent_executor.stream(
    {
        "user_input": "네이버에서 자체 개발한 생성형 AI 관련된 정보를 문서에서 찾아주세요."
    } # 구어체로 input을 주어도 아래 query 를 확인하면 정확한 키워드를 찾아서 검색을 시도
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별 출력
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: 네이버 생성형 AI
Log: 
Invoking: `pdf_search` with `{'query': '네이버 생성형 AI'}`



[관찰 내용]
Observation: ㅇ 주요 AI 모델 저작권 침해 평가에 따르면, 오픈AI의 GPT-4가 100개의 의도적 프롬프트에 대해 44%의 답변에서 책 내용을 정확하게 복제한 내용 생성 (2024.3)ㅇ 엔비디아의 생성 AI 플랫폼 ‘네모’가 소설 저작권을 침해했다고 기소하는 등 데이터 저작권에 대한 분쟁 확대 (2024.3)ㅇ 미국 저작권청은 미드저니를 통해 생성한 작품인 ‘새벽의 자리야’는 작품 자체에 대한 저작권은 인정하지 않고, AI로 만든 이미지와 텍스트를 조정한 것에 대해서만 인정 (2023.9)
43 Nasr 외(2023.11), Scalable Extraction of Training D ata from  (Production) Language M odels, arXiv:2311.17035v1

ㅇ KB국민, 하나, 신한은행 등 금융 분야를 비롯하여 다양한 분야에서의 AI 상담 서비스(콜센터)의 도입으로 AI가 상담사의 역할을 대체 (2024.2.17)38ㅇ 개발자의 단순 코딩 업무 대체, 제조 분야에서의 공정 자동화, 문서 작성 등 AI는 높은 생산성을 제공하여 사람보다 더 효율적인 작업이 가능 (2024.2.17)ㅇ 포스코는 무인로봇·비전AI 등을 활용한 공정 자동화로 인텔리전트 팩토리로의 전환을 실현하여 최소한의 인력만 유지39 (2024.6.18)사진: American Banker(2018)40■(글로벌AI 격차) 고급AI 개발자원의보유여부에따른국가간격차발생●고급 AI 개발을 위한 다양한 필요조건들로 인해 AI 빅테크 기업들의 지배력 확대∙디지털 기술 활용률, 컴퓨팅 자원 접근성, 인프라, 경제적 의존도 등은 범용 AI 개발 및 배포에 있어 중요한 요소로서, 이로 인해 범용 AI의 개발은 현재 서구 국가 및 중국에 집중된 경향*특히 미국에 

### 이전 대화를 기억하는 Agent

이전 대화를 기억하기 위해서는 `RunnableWithMessageHistory` 를 사용하고 `AgentExcutor` 를 감싸줍니다.  
- [RunnableWithMessageHistory](https://wikidocs.net/254682)

In [49]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# session_id 를 저장할 딕셔너리 생성
store = {}

# session_id 를 기반으로 세션 기록을 가져오는 함수, store - 기억 저장소
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        store[session_ids] = ChatMessageHistory() # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트 생성
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor, # agent_excutor
    get_session_history, # 대화 session_id
    input_messages_key="user_input", # 프롬프트의 질문이 입력되는 key: "user_input"
    history_messages_key="chat_history", # 프롬프트의 메시지가 입력되는 key: "chat_history"
)

In [54]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
response = agent_with_chat_history.stream(
    {"user_input": "네이버가 개발한 생성형 AI 관련된 정보를 문서에서 찾아주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "agent1"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: Naver generative AI
Log: 
Invoking: `pdf_search` with `{'query': 'Naver generative AI'}`



[관찰 내용]
Observation: ㅇ 주요 AI 모델 저작권 침해 평가에 따르면, 오픈AI의 GPT-4가 100개의 의도적 프롬프트에 대해 44%의 답변에서 책 내용을 정확하게 복제한 내용 생성 (2024.3)ㅇ 엔비디아의 생성 AI 플랫폼 ‘네모’가 소설 저작권을 침해했다고 기소하는 등 데이터 저작권에 대한 분쟁 확대 (2024.3)ㅇ 미국 저작권청은 미드저니를 통해 생성한 작품인 ‘새벽의 자리야’는 작품 자체에 대한 저작권은 인정하지 않고, AI로 만든 이미지와 텍스트를 조정한 것에 대해서만 인정 (2023.9)
43 Nasr 외(2023.11), Scalable Extraction of Training D ata from  (Production) Language M odels, arXiv:2311.17035v1

ㅇ 국방 분야의 워게임 시뮬레이션에 생성 AI를 적용했을 때 사용자가 예측하기 어려운 전술 의사결정을 보이며, 드물게 고위험성의 핵무기 배치까지 이어지는 것으로 확인(2024.1.7)33ㅇ 미국의 프로젝트 메이븐, 중국의 인민해방군 AI 사령관, 이스라엘의 라벤더는 군사용 AI로 실전에 투입되어, 시뮬레이션, 공격대상 타격, 적 탐지 등의 용도로 활용29 Jinzhe Tan 외 (2023.7), "ChatGPT as an Artificial Lawyer?" 30 Yixin Wan 외 (2023.12), ““Kelly is a Warm Person, Joseph is a Role Model”: Gender Biases in LLM-Generated Reference Letters“31 Kelly Avery Mack 외 (2024.5), ““They only care to show us the wheelchair”: disabi

In [55]:
response = agent_with_chat_history.stream(
    {"user_input": "이전의 답변을 영어로 번역해 주세요."},
    config={"configurable": {"session_id": "agent1"}}, # 기억된 정보를 불러오기위해 세션아이디를 동일하게 설정
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: Naver generative AI
Log: 
Invoking: `pdf_search` with `{'query': 'Naver generative AI'}`



[관찰 내용]
Observation: ㅇ 주요 AI 모델 저작권 침해 평가에 따르면, 오픈AI의 GPT-4가 100개의 의도적 프롬프트에 대해 44%의 답변에서 책 내용을 정확하게 복제한 내용 생성 (2024.3)ㅇ 엔비디아의 생성 AI 플랫폼 ‘네모’가 소설 저작권을 침해했다고 기소하는 등 데이터 저작권에 대한 분쟁 확대 (2024.3)ㅇ 미국 저작권청은 미드저니를 통해 생성한 작품인 ‘새벽의 자리야’는 작품 자체에 대한 저작권은 인정하지 않고, AI로 만든 이미지와 텍스트를 조정한 것에 대해서만 인정 (2023.9)
43 Nasr 외(2023.11), Scalable Extraction of Training D ata from  (Production) Language M odels, arXiv:2311.17035v1

ㅇ 국방 분야의 워게임 시뮬레이션에 생성 AI를 적용했을 때 사용자가 예측하기 어려운 전술 의사결정을 보이며, 드물게 고위험성의 핵무기 배치까지 이어지는 것으로 확인(2024.1.7)33ㅇ 미국의 프로젝트 메이븐, 중국의 인민해방군 AI 사령관, 이스라엘의 라벤더는 군사용 AI로 실전에 투입되어, 시뮬레이션, 공격대상 타격, 적 탐지 등의 용도로 활용29 Jinzhe Tan 외 (2023.7), "ChatGPT as an Artificial Lawyer?" 30 Yixin Wan 외 (2023.12), ““Kelly is a Warm Person, Joseph is a Role Model”: Gender Biases in LLM-Generated Reference Letters“31 Kelly Avery Mack 외 (2024.5), ““They only care to show us the wheelchair”: disabi

### Agent Template

템플릿 코드로서 프로젝트를 진행할때 복사기 편하게 한 셀에 모아두었습니다. 

In [57]:
# 필요한 모듈 import
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.document_loaders import PyMuPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_print import AgentStreamParser

########## 1. 도구 정의##########

### 1-1. Search 도구 ###
# TavilySearchResults 클래스의 인스턴스를 생성
# k : 검색 결과 갯수
search = TavilySearchResults(k=6)

### 1-2. PDF 문서 검색 도구 (Retriever) ###
# PDF 파일 로드. 파일의 경로 입력
loader = PyMuPDFLoader("./data/IS-183_AI 위험 유형 및 사례 분석(최종).pdf")

# 텍스트 분할기를 사용하여 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드, text_spllitter에 맞게 분할
split_docs = loader.load_and_split(text_splitter)

# VectorStore 생성
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever 생성
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구 이름 설정
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명, 자세히 기입하는 것이 성능에 좋음
)

### 1-3. tools 정의, 리스트에 도구 목록을 추가###
# tools 리스트에 search와 retriever_tool을 추가
tools = [search, retriever_tool]

########## 2. LLM 정의 ##########
# LLM 모델 생성
llm = ChatOpenAI(model="gpt-4o", temperature=0)

########## 3. Prompt 정의 ##########

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `web_search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{user_input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

########## 4. Agent 정의 ##########

# create_tool_calling_agent() 함수를 사용하여 agent 정의, llm, tools, prompt 를 연결
# llm, tools, prompt를 인자로 사용합니다.
agent = create_tool_calling_agent(llm, tools, prompt)

########## 5. AgentExecutor 정의 ##########

# AgentExecutor 클래스를 사용하여 agent와 tools 설정
# 상세한 로그를 출력하도록 verbose를 True로 설정
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

########## 6. 채팅 기록을 수행하는 메모리 추가 ##########

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor, # agent_executor 설정
    get_session_history, # 대화 session_id
    input_messages_key="user_input", # 프롬프트의 질문이 입력되는 key: "input"
    history_messages_key="chat_history", # 프롬프트의 메시지가 입력되는 key: "chat_history"
)

########## 7. Agent 파서 정의 ##########
agent_stream_parser = AgentStreamParser()

In [59]:
########## 8. 에이전트 실행, 결과 확인 ##########

response = agent_with_chat_history.stream(
    {"user_input": "AI의 위험 유형에 대해 간략하게 소개해주세요."},
    config={"configurable": {"session_id": "sessionid:result1"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: AI의 위험 유형
Log: 
Invoking: `pdf_search` with `{'query': 'AI의 위험 유형'}`



[관찰 내용]
Observation: SPRi이슈리포트IS-183
AI위험유형및사례분석
5
■현재의연구들은다양한관점과범위에따라AI위험요인을정의하고있어,본장에서는
네가지주요연구자료에서제시하고있는위험과위험요인들을비교하고분석을수행
●현재 논의되고 있는 AI 위험 요인들은 다양한 기준에 의해 수집 및 분류되어 제시되고 있어, AI 
위험에 관한 포괄적 분류체계 마련을 위해 비교·분석이 필요
∙AI 위험 요인들은 유사하지만 다양한 정의와 분류에 따라 혼재하고 있고, 각 위험 요인의 대응 방안에 관한 
해결 방식도 다른 경향을 보여, 혼재된 위험 요인들을 같은 수준에서 비교할 필요가 있음 
* 미국 NIST의 생성 AI 프로파일(2024.7), MIT 연구진의 AI 위험 분류(2024.8), 중국 연구진의 AI 위험 분류(2024.6) 등
■(연구1)영국에서는AI안전성정상회의이후,AI의안전에관한최초의국제과학보고서가
발표되었고,위험유형과위험완화를위한기술적접근방식등주요내용을포함12
●세계적인 인공지능 전문가 요슈아 벤지오 교수의 책임 하에 다양한 AI 전문가들이 참여하여 
6개월 동안 수행한 연구로, 향후 예상되는 첨단 AI의 기능과 위험, 기술적 대응 방안에 관한 
다양한 의견을 수렴
●범용 AI에 초점을 두고, 범용 AI가 유발할 수 있는 AI 위험을 △ 악의적 사용 위험, △ 오작동 위험 
△ 시스템적 위험 △ 교차(cross-cutting) 위험 요인의 4가지로 구분하여 제시
∙(악의적 사용 위험) 범용 AI는 광범위한 지식 영역을 다루는 만큼, 악의적인 목적으로 기존 용도가 변경되어 
사용될 위험이 존재
∙(오작동 위험) 기능에 대한 오해, 부적절한 지침, 편향된 학습 데이터와 결함이 있는 시스템의 배포 등으로 
오작동의 위험이 증가
∙(시스템적 위험)  AI 활용 증

In [61]:
########## 8. 에이전트 실행, 결과 확인 ##########

response = agent_with_chat_history.stream(
    {"user_input": "이전의 답변을 영어로 번역해 주세요"},
    config={"configurable": {"session_id": "sessionid:result1"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
Here is the translation of the previous answer into English:

The types of AI risks are categorized in various ways across different studies. The main types of risks include:

1. **Malicious Use Risk**: This refers to the potential for AI to be used for malicious purposes. It includes the risk of AI's extensive knowledge being exploited.

2. **Malfunction Risk**: This involves the risk of malfunctions due to misunderstandings of AI system functions, inappropriate instructions, biased training data, and the deployment of defective systems.

3. **Systemic Risk**: This includes negative impacts from social, economic, and ethical perspectives due to the increased use of AI.

4. **Cross-Risk Factors**: This encompasses various risks that general AI might pose, including technical elements, non-technical aspects of development and deployment, and social risk factors.

These types of risks play a crucial role in identifying and establishing response measures by AI safety agencies and 

In [66]:
########## 8. 에이전트 실행, 결과 확인 ##########

response = agent_with_chat_history.stream(
    {
        "user_input": "스타벅스 2024 winter 프리퀀시 미션음료는 무엇인가요?"
    },
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "sessionid:result2"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 스타벅스 2024 winter 프리퀀시 미션음료
Log: 
Invoking: `tavily_search_results_json` with `{'query': '스타벅스 2024 winter 프리퀀시 미션음료'}`



[관찰 내용]
Observation: [{'url': 'https://m.post.naver.com/viewer/postView.naver?volumeNo=40042593&memberNo=1758487', 'content': '스타벅스 2024 winter e-frequency 이벤트는 11월 1일부터 12월 31일까지 진행됩니다. 겨울 시즌 동안 따뜻한 스타벅스 음료와 함께 특별한 증정품을 즐기며 추운 겨울을 감성적으로 보내 보세요. #프리퀀시 #미션음료 #스타벅스미션음료 #스타벅스프리퀀시 #frequency #'}, {'url': 'https://m.blog.naver.com/hxxxx_u/223630058429', 'content': '2024 스타벅스 겨울 e-프리퀀시 총정리 : 네이버 블로그 본문 바로가기 카테고리 이동 님의 블로그 2024 스타벅스 겨울 e-프리퀀시 총정리 이웃추가 본문 기타 기능 공유하기 2024 스타벅스 겨울 e-프리퀀시가 스타벅스 2025 플래너 데일리 플래너 2종 (그린, 핑크) 언데이티드 플래너 1종 (브라운) 3.글로벌 필기앱 굿노트(Good Notes)와 협업 디지털 플래너 1종 (앱 제공) 스타벅스 포터블 램프 4종 디자인 조명 전문 브랜드 보나키아와 협업 스타벅스 2025 캘린더 이희조 작가와 협업 e-프리퀀시 미션 음료 겨울 -프리퀀시 미션 음료는 11월 1일 출시되는 크리스마스 시즌 음료 4.스타벅스 딸기 라떼 리저브 카테고리 e-프리퀀시 이용방법 플래너, 포터블 램프, 캘린더 중 1종 증정 e-프리퀀시 기간 및 예약 방법 11월 1일부터 12월 31일까지 스타벅스 앱에서 가능해요 #스타벅스 댓글 50 공유하기 {"

In [68]:
########## 8. 에이전트 실행, 결과 확인 ##########

response = agent_with_chat_history.stream(
    {"user_input": "이전의 답변을 SNS 게시글 형태로 100자 내외로 작성하세요."},
    config={"configurable": {"session_id": "sessionid:result2"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
스타벅스 2024 겨울 프리퀀시! 미션 음료: 논알콜 홀리데이 패션 티 뱅쇼, 딸기 라떼, 얼그레이 바닐라 티 라떼, 리저브 음료. 3잔 포함 총 17잔으로 프리퀀시 완성!


In [71]:
########## 8. 에이전트 실행, 결과 확인 ##########

response = agent_with_chat_history.stream(
    {"user_input": "이전의 답변에 스타벅스 2024 winter 증정품을 추가하세요."},
    config={"configurable": {"session_id": "sessionid:result2"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: tavily_search_results_json
query: 스타벅스 2024 winter 프리퀀시 증정품
Log: 
Invoking: `tavily_search_results_json` with `{'query': '스타벅스 2024 winter 프리퀀시 증정품'}`



[관찰 내용]
Observation: [{'url': 'https://m.post.naver.com/viewer/postView.naver?volumeNo=40042593&memberNo=1758487', 'content': '2024년 스타벅스 겨울 프리퀀시 이벤트 증정품 3개 및 미션음료 참가방법 ... 스타벅스 2024 winter e-frequency 이벤트는 11월 1일부터 12월 31일까지 진행됩니다. 겨울 시즌 동안 따뜻한 스타벅스 음료와 함께 특별한 증정품을 즐기며 추운 겨울을 감성적으로 보내 보세요.'}, {'url': 'https://icelattematcha.tistory.com/5', 'content': '[스타벅스 2024 winter e-frequency 증정품 안내] 2024 윈터 이프리퀀시 윈터 프리퀀시 2024.11.07 01:02 [스타벅스 리유저블백] [투고 사이렌 리유저블 백] 시그니처 요소를 담은 리유저블 백 출시! 2024.11.01 15:19'}, {'url': 'https://blog.naver.com/PostView.naver?blogId=enrong912&logNo=223642696366', 'content': '블로그 블로그 검색 블로그 메뉴 블로그  블로그 스타벅스 2024년 WINTER e-프리퀀시 시작 /증정품 종류 및 수령 방법 출처-스타벅스 다시 스타벅스 e-프리퀀시의 계절이 돌아왔네요. 미션음료 3잔 포함 총 17잔의 음료를 구매하시면, 스타벅스 플래너, 스타벅스 캘린더, 스타벅스 포터블 램프 중 1개를 증정 스타벅스 모바일 APP 또는 홈페이지(PC 및 모바일)에 등록된 -프리퀸시 바코드 화면 또는 스타벅